## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-20-10-53-21 +0000,bbc,When will the Epstein files be released and wi...,https://www.bbc.com/news/articles/cq50nvq8znvo...
1,2025-11-20-10-38-54 +0000,bbc,Burnham refuses to rule out challenging Starme...,https://www.bbc.com/news/articles/cj9797pvmk7o...
2,2025-11-20-10-36-00 +0000,wsj,"Nvidia Profits Soar, Soothing Investor Jitters...",https://www.wsj.com/tech/ai/nvidia-earnings-q3...
3,2025-11-20-10-34-41 +0000,nyt,Nvidia Earnings Fuel Rally in Global Technolog...,https://www.nytimes.com/2025/11/19/business/st...
4,2025-11-20-10-31-47 +0000,nyt,Mount Semeru Erupts and Rains Ash on Villages,https://www.nytimes.com/2025/11/20/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2338/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
201,trump,41
82,house,21
2,epstein,21
167,new,17
487,saudi,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
114,2025-11-20-01-59-00 +0000,wsj,President Trump signed legislation authorizing...,https://www.wsj.com/politics/policy/trump-sign...,118
100,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...,118
72,2025-11-20-05-14-10 +0000,startribune,Trump signs bill to release Jeffrey Epstein ca...,https://www.startribune.com/trump-signs-bill-t...,109
68,2025-11-20-06-05-03 +0000,bbc,Trump signs bill ordering US justice departmen...,https://www.bbc.com/news/articles/cly2y40lj5go...,108
127,2025-11-20-01-00-00 +0000,wsj,By feting Saudi Crown Prince Mohammed bin Salm...,https://www.wsj.com/politics/policy/trump-puts...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
114,118,2025-11-20-01-59-00 +0000,wsj,President Trump signed legislation authorizing...,https://www.wsj.com/politics/policy/trump-sign...
192,80,2025-11-19-21-43-13 +0000,nypost,"Elon Musk, Nvidia unveil deal with Saudi state...",https://nypost.com/2025/11/19/business/elon-mu...
100,77,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...
117,51,2025-11-20-01-41-44 +0000,wapo,U.S. pushes new Ukraine peace plan as Trump’s ...,https://www.washingtonpost.com/national-securi...
286,50,2025-11-19-16-32-56 +0000,nypost,"Trump Media stock crashes to all-time lows, wi...",https://nypost.com/2025/11/19/business/trump-m...
89,43,2025-11-20-03-30-00 +0000,wsj,The House Ethics Committee said Wednesday it w...,https://www.wsj.com/politics/ethics-probe-of-r...
213,41,2025-11-19-20-41-11 +0000,nyt,Israeli Strikes Kill at Least 25 in Gaza Amid ...,https://www.nytimes.com/2025/11/19/world/middl...
70,40,2025-11-20-05-45-10 +0000,nypost,"Ex-CT school employee texted boy, 11, nearly 5...",https://nypost.com/2025/11/20/us-news/connecti...
127,38,2025-11-20-01-00-00 +0000,wsj,By feting Saudi Crown Prince Mohammed bin Salm...,https://www.wsj.com/politics/policy/trump-puts...
322,34,2025-11-19-14-36-28 +0000,cbc,Former U.S. treasury secretary Larry Summers t...,https://www.cbc.ca/news/world/larry-summers-ep...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
